In [2]:
%run PrimeItemParser.ipynb

In [3]:
import requests
import json
import os
import time
from datetime import datetime
import glob

def jprint(text):
    print(json.dumps(text, indent=4))
    
#1 API Call
def getBestTraders(itemname):
    url = f'https://api.warframe.market/v1/items/{itemname}/orders'
    text = requests.get(url).json()
    text = text["payload"]["orders"]
    sellers = [order for order in text if order["order_type"] == "sell" and order["user"]["status"] == "ingame" and order["user"]["region"] == "en" and order["platform"] == "pc"]
    buyers = [order for order in text if order["order_type"] == "buy" and order["user"]["status"] == "ingame" and order["user"]["region"] == "en" and order["platform"] == "pc"]
    bestSeller = sorted(sellers, key=lambda order: order["platinum"])[0]
    bestBuyer = sorted(buyers, key=lambda order: order["platinum"], reverse=True)[0]
    #jprint(bestseller)
    return bestSeller, bestBuyer

#Returns data for the bestSeller and bestBuyer
def getMarketSnapshot(itemname):
    bestSeller, bestBuyer = getBestTraders(itemname)
    return {"datetime" : datetime.now().isoformat(sep=" ", timespec="seconds"), "BuyerPrice": bestBuyer["platinum"], "SellerPrice": bestSeller["platinum"]}

#Clears all the data in every dataframe recorded between startDate and endDate
def clearDataFrames(primesToAnalyze):
    for prime in primesToAnalyze:
        emptyDF = pd.DataFrame(columns=['datetime', 'BuyerPrice', 'SellerPrice'])
        emptyDF.to_csv(f"itemDataFrames/{prime}.csv")

#jprint(getBestTraders("ash_prime_set")[0])


In [4]:
import pandas as pd
import os

def initDataFrames(primesToAnalyze):
    existingDataFrames = os.listdir("itemDataFrames")
    for prime in primesToAnalyze:
        if not f"{prime}.csv" in existingDataFrames:
            emptyDF = pd.DataFrame(columns=['datetime', 'BuyerPrice', 'SellerPrice'])
            emptyDF.to_csv(f"itemDataFrames/{prime}.csv")


In [5]:
#clearDataFrames(primeSetNames)

In [ ]:
import time
from IPython.display import clear_output


def updateCSVs(primesToAnalyze):
    for i, primeName in enumerate(primesToAnalyze):
        primeDF = pd.read_csv(f"itemDataFrames/{primeName}.csv", index_col=0)
        currentPrices = pd.DataFrame(getMarketSnapshot(primeName), index=[0])
        primeDF = pd.concat([primeDF, currentPrices], ignore_index=True) 
        primeDF.to_csv(f"itemDataFrames/{primeName}.csv")
        if i % 3 == 0:
            print(i)
            time.sleep(1.1)
            
%run PrimeItemParser.ipynb
primesToAnalyze = primeSetNames

initDataFrames(primesToAnalyze)

i = 0
while True:
    clear_output(wait=True)
    print("Iteration", i)
    updateCSVs(primesToAnalyze)
    i += 1


Iteration 108
0
3
6
9
12
15
18
21
24
27
30
